In [1]:
import numpy as np
from upper_envelope.fues_numba.fues_numba import fast_upper_envelope_wrapper
import numba as nb
from collections import namedtuple


In [2]:
@nb.njit
def utility_crra(consumption: np.array, params_ntuple) -> np.array:
    """Computes the agent's current utility based on a CRRA utility function.

    Args:
        consumption (jnp.array): Level of the agent's consumption.
            Array of shape (i) (n_quad_stochastic * n_grid_wealth,)
            when called by :func:`~dcgm.call_egm_step.map_exog_to_endog_grid`
            and :func:`~dcgm.call_egm_step.get_next_period_value`, or
            (ii) of shape (n_grid_wealth,) when called by
            :func:`~dcgm.call_egm_step.get_current_period_value`.
        choice (int): Choice of the agent, e.g. 0 = "retirement", 1 = "working".
        params_dict (dict): Dictionary containing model parameters.
            Relevant here is the CRRA coefficient theta.

    Returns:
        utility (jnp.array): Agent's utility . Array of shape
            (n_quad_stochastic * n_grid_wealth,) or (n_grid_wealth,).

    """
    utility_consumption = (consumption ** (1 - params_ntuple.rho) - 1) / (
        1 - params_ntuple.rho
    )

    utility = utility_consumption - (1 - params_ntuple.choice) * params_ntuple.delta

    return utility

In [3]:
max_wealth = 50
n_grid_wealth = 500
exog_savings_grid = np.linspace(0, max_wealth, n_grid_wealth)

beta = 0.95  # discount_factor

utility_kwargs = {
    "choice": 0,
    "rho": 1.95,
    "delta": 0.35,
}

In [4]:
resource_dir = "../../tests/resources/"
value_egm = np.genfromtxt(
    resource_dir + "upper_envelope_period_tests/val2.csv",
    delimiter=",",
)
policy_egm = np.genfromtxt(
    resource_dir + "upper_envelope_period_tests/pol2.csv",
    delimiter=",",
)
n_constrained_points_to_add = int(0.1 * len(policy_egm[0]))
n_final_wealth_grid = int(1.2 * (len(policy_egm[0])))
tuning_params = {
    "n_final_wealth_grid": n_final_wealth_grid,
    "jump_thresh": 2,
    "n_constrained_points_to_add": n_constrained_points_to_add,
    "n_points_to_scan": 10,
}

In [5]:

utility_ntuple = namedtuple("utility_params", utility_kwargs.keys())(
    *utility_kwargs.values()
)
tuning_params_tuple = namedtuple("tunings", tuning_params.keys())(
    *tuning_params.values()
)

In [6]:
endog_grid_refined, policy_refined, value_refined = fast_upper_envelope_wrapper(
    endog_grid=policy_egm[0, 1:],
    policy=policy_egm[1, 1:],
    value=value_egm[1, 1:],
    expected_value_zero_savings=value_egm[1, 0],
    exog_grid=exog_savings_grid,
    utility_function=utility_crra,
    utility_kwargs=utility_ntuple,
    discount_factor=beta,
    tuning_params=tuning_params_tuple,
)